# Interview Tool

This is an automated interview system. It will ask a question and then record the answer. This answer will be transcribed to text using ASR, summarised and traslated to Malay.

In [1]:
import os
import riva.client
auth = riva.client.Auth(uri='10.100.182.246:50051')

In [2]:
asr_service = riva.client.ASRService(auth)

offline_config = riva.client.RecognitionConfig(
    encoding=riva.client.AudioEncoding.LINEAR_PCM,
    max_alternatives=1,
    enable_automatic_punctuation=True,
    verbatim_transcripts=False,
)

In [3]:
questions = ["What is your name?", "What degree have to studied?", "Tell me about one of your hobbies?", "Why do you want to work for us?"]
answers = []

Boosting allows Livs to be detected

In [4]:
boosted_lm_words = ['Livs']
boosted_lm_score = 20.0

In [5]:
wav_filedir = "answers"
wav_filenames = ["Liwia_name.wav", "Liwia_study.wav", "Liwia_hobby.wav", "Liwia_work.wav"]
for wav_name in wav_filenames:
    wav_path = os.path.join(wav_filedir, wav_name)
    riva.client.add_audio_file_specs_to_config(offline_config, wav_path)
    with open(wav_path, "rb") as f:
        data = f.read()
    response = asr_service.offline_recognize(data, offline_config)
    answers.append(str(response.__repr__().split("\"")[1:2])[2:-2])

In [6]:
for id, answer in enumerate(answers):
    print(questions[id])
    print(answer)
    print("\n")

Hello, my name is Livia, but please call me Lives. 


I am currently studying theoretical physics at the University of Bristol. This year I will be finishing my final year and completing my Master\\'s project. 


am a keen scuba diver and indoor climber. I am currently in a committee role in the largest scuba diving club in the U. K. really enjoy wreck diving, especially when you can penetrate the wreck. They are also really good for marine wildlife as they become artificial reefs. Last September I went for my 1st scuba trip abroad. It was a lot of fun. We went diving in Gozo, Malta. The water was much warmer and the visibility was approximately 30 m. It was a crazy experience to be a 35 m with a fish and be able to look up and still see the surface. This was nothing like the Uk, where sometimes you cannot even see your dive buddy. 


I love coding and problem solving, so I think I would make a great addition to your team. I am also very interested in the idea of moving to Malaysia as 

# SUMMERIZES TEXT

In [7]:
import numpy as np
import tritonclient.http as httpclient
import riva.client
auth = riva.client.Auth(uri='10.100.182.246:50051')

from tritonclient.utils import np_to_triton_dtype

URL = "10.98.96.143:8000"
MODEl_GPTJ_FASTERTRANSFORMER = "ensemble" 

OUTPUT_LEN = 128
BATCH_SIZE = 1
BEAM_WIDTH = 2
TOP_K = 1
TOP_P = 0.0

start_id = 220
end_id = 50256

In [8]:
client = httpclient.InferenceServerClient(URL,
                                           concurrency=1,
                                           verbose=False)

In [9]:
# Inference hyperparameters
def prepare_tensor(name, input):
    tensor = httpclient.InferInput(
        name, input.shape, np_to_triton_dtype(input.dtype))
    tensor.set_data_from_numpy(input)
    return tensor

# explanation
def prepare_inputs(input0):
    bad_words_list = np.array([[""]], dtype=object)
    stop_words_list = np.array([[""]], dtype=object)
    input0_data = np.array(input0).astype(object)
    output0_len = np.ones_like(input0).astype(np.uint32) * OUTPUT_LEN
    runtime_top_k = (TOP_K * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    runtime_top_p = TOP_P * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    beam_search_diversity_rate = 0.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    temperature = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    len_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    repetition_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    random_seed = 0 * np.ones([input0_data.shape[0], 1]).astype(np.int32)
    is_return_log_probs = True * np.ones([input0_data.shape[0], 1]).astype(bool)
    beam_width = (BEAM_WIDTH * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    start_ids = start_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)
    end_ids = end_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)

    inputs = [
        prepare_tensor("INPUT_0", input0_data),
        prepare_tensor("INPUT_1", output0_len),
        prepare_tensor("INPUT_2", bad_words_list),
        prepare_tensor("INPUT_3", stop_words_list),
        prepare_tensor("runtime_top_k", runtime_top_k),
        prepare_tensor("runtime_top_p", runtime_top_p),
        prepare_tensor("beam_search_diversity_rate", beam_search_diversity_rate),
        prepare_tensor("temperature", temperature),
        prepare_tensor("len_penalty", len_penalty),
        prepare_tensor("repetition_penalty", repetition_penalty),
        prepare_tensor("random_seed", random_seed),
        prepare_tensor("is_return_log_probs", is_return_log_probs),
        prepare_tensor("beam_width", beam_width),
        prepare_tensor("start_id", start_ids),
        prepare_tensor("end_id", end_ids),
    ]
    return inputs

In [10]:
def convert(s):
    # initialization of string to ""
    new = ""
    # traverse in the string
    for x in s:
        new += x
    # return string
    return [new]

one = 'particle_phys.txt'
two = 'telephone.txt'
three = 'tree.txt'

f = open(three, "r")
longer_inputs = []

for x in f:
    longer_inputs.append(x)

temp = [""]
for i in longer_inputs:
    if i != '\n':
        temp += i
        

temp2 = convert(temp)
temp2.append('\n\ntl;dr:\n')

In [11]:
input0 = [[temp2[0] + temp2[1]],]
inputs = prepare_inputs(input0)
result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
output0 = result.as_numpy("OUTPUT_0")

In [12]:
print(output0[0].decode('UTF-8'))

A tree is a tall plant with a trunk and branches made of wood. Trees can live for many years. The oldest tree ever discovered is approximately 5,000 years old and the oldest tree from the UK is about 1,000. The four main parts of a tree are the roots, the trunk, the branches, and the leaves.
The roots of a tree are usually under the ground. However, this is not always true. The roots of the mangrove tree are often under water. A single tree has many roots. The roots carry nutrients and water from the ground through the trunk and branches to the leaves of the tree. They can also breathe in air. Sometimes, roots are specialized into aerial roots, which can also provide support, as is the case with the banyan tree.
The trunk is the main body of the tree. The trunk is covered with bark which protects it from damage. Branches grow from the trunk. They spread out so that the leaves can get more sunlight.
The leaves of a tree are green most of the time, but they can come in many colors, shape

# Translation

In [62]:
def Translate( User_input, input_lang):
    
    if input_lang == 'EN' :
        
        Training_phrases = (  
        "English: The horse looks sexy "
        "Malay: Kuda itu kelihatan seksi"

        "English: your father is a sexy policeman. "
        "Malay: ayah kamu merupakan seorang polis yang seksi."

        "English: I like cows. "
        "Malay: Saya suka lembu."

        "English: This company provide great opportunity. "
        "Malay: syarikat ini memberikan peluang yang besar."

        "English: I love coral reefs"
        "Malay: saya suka terumbu karang"

        "English: Albert Einstein is a theoretical physicist"
        "Malay: Albert Einstein ialah seorang ahli fizik teori"
        
        "English: he is part of the golf club committee"
        "Malay: dia adalah sebahagian daripada jawatankuasa kelab golf"
        
        )

        Lang_en = "English:"
        True_input = Lang_en + User_input

        input0 = [[Training_phrases + True_input ],]
        inputs = prepare_inputs(input0)
        result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
        output0 = result.as_numpy("OUTPUT_0")

        output0 = np.array2string(output0)

        k = output0.find(User_input)

        k2 = output0.find('Malay', k)

        m1 = output0.find( 'English',(k+ len(User_input ) ))
        offset = k + len('Malay') + len(User_input)
        m2 = output0.find( 'Malay',(offset )) #these three line is here because the AI will spit nonesense
        #after it is done translating, so we have to pick it up when the AI 

        m = min(m1,m2)


        print(output0[k2:m])


            

    if input_lang == 'BM' :

        Training_phrases = (
        "Malay: Di mana stesen kereta api terdekat?"
        "English: Where is the nearest train station? "

        "Malay: Berapakah harga tiket kereta api ke warsaw? "
        "English: How much is a train ticket to Warsaw?"

        "Malay: Jepun merupakan sesuatu bandar di dalam Tokyo"
        "English: Japan is a city in Tokyo"

        "Malay: Hang Tuah ialah seorang pahlawan legenda berbangsa Melayu pada masa pemerintahan Kesultanan Melayu Melaka di abad ke-15 "
        "English: Hang Tuah was a legendary warrior of the Malay race during the reign of the Malacca Sultanate in the 15th century "

        "Malay: Bolehkah anda cakap bahasa melayu?"
        "English: can you speak malay?"
        
    
    
        )
        
        Lang_bm = "Malay:"
        True_input = Lang_bm + User_input

        input0 = [[Training_phrases + True_input ],]
        inputs = prepare_inputs(input0)
        result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
        output0 = result.as_numpy("OUTPUT_0")

        output0 = np.array2string(output0)
        k = output0.find(User_input)

        k2 = output0.find('English', k)

        m1 = output0.find( 'Malay',(k+ len(User_input ) ))
        offset = k + len('English') + len(User_input)
        m2 = output0.find( 'English',(offset )) #these three line is here because the AI will spit nonesense
        #after it is done translating, so we have to pick it up when the AI 

        m = min(m1,m2)
        
        
        print(output0[k2:m])
                    
        
    

In [63]:
questions = ["What is your name?", "What degree have to studied?", "Tell me about one of your hobbies?", "Why do you want to work for us?"]
print(response.__repr__().split("\"")[1:2])

['I love coding and problem solving, so I think I would make a great addition to your team. I am also very interested in the idea of moving to Malaysia as the weather looks much better over there. ']


In [ ]:
for sent in y:
    Translate(sent, "EN")

Malay: saya adalah penggemar kipas dan penggemar ruang klima.
Malay: saya saat ini berada dalam komite yang paling besar dalam klub penyelamat karang di Amerika Serikat.

Malay: saya benar-benar menikmati pengungsi, terutamanya ketika anda dapat melepaskan kapal
Malay: mereka juga sangat bagus untuk hewan laut karena mereka menjadi bukit ilegal.\n\n
Malay: tahun lalu saya pergi untuk pertama kalinya scuba di luar negeri.
Malay: itu banyak menyenangkan.
Malay: kami pergi berjalan-jalan di Gozo, Malta.
Malay: air itu jauh lebih hangat dan visibilitasnya kira-kira 30 m.


